### In-Game Purchase Data Analysis

#### Background 

In this homework, you are ask to deal with a task of analyzing an "in-game purchase" dataset. In this game, players are able to to purchase optional items that enhance their playing experience. Now your task is to generate a report that breaks down the game's purchasing data into meaningful insights.

-----

#### Observable Trends

* There are 1163 active players. The vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.79%) with secondary groups falling between 15-19 (18.58%) and 25-29 (13.37%). 

* The age group that spends the most money is the 20-24 with 1,114.06 dollars as total purchase value and an average purchase of 4.32. In contrast, the demographic group that has the highest average purchase is the 35-39 with 4.76 and a total purchase value of 147.67. 
-----


In [2]:
import pandas as pd
import numpy as np

# Raw data file
file_to_load = "purchase_data.csv"

# Read purchasing file and store into pandas data frame
purchase_data = pd.read_csv(file_to_load)

In [3]:
data = purchase_data.copy()
data.head(3)
#使用copy，若之後需要遭修改的原始dataframe則不用重新load

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88


## Player Count

* Display the total number of players


In [4]:
total = len(set(data.SN))
#使用set避免重複計算
pd.DataFrame([total] , columns=['Total players'])

,Total players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.
* Create a summary data frame to hold the results
* Display the summary data frame

In [5]:
num_of_uni_item = len(set(data['Item ID']))
ave_price = '$'+'%3.2f'%data['Price'].mean()
#因需要錢字符，將後面的數字設為string
num_of_pur = data.shape[0]
total_rev = '$'+str(data['Price'].sum())
pd.DataFrame([num_of_uni_item , ave_price , num_of_pur , total_rev],['Number of Unique Items' , 'Average Price' , 'Number of Purchases' , 'Total Revenue'],).transpose()

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,$2379.77


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [6]:
data_unique_name = data.drop_duplicates('SN')
#使用drop_duplicates避免SN重複出現
per= 100*(data_unique_name.groupby('Gender').count()/data_unique_name.shape[0])['Purchase ID']
#百分比為當性別總數/全性別加總
total = data_unique_name.groupby('Gender').count()['Purchase ID']
ans = pd.DataFrame({'Percentage of Players':per , 'Total Count':total} , index = ['Male' , 'Female' , 'Other / Non-Disclosed']).round(2)
ans.index.name = 'Gender'
ans

,Percentage of Players,Total Count
Gender,,
Male,84.03,484
Female,14.06,81
Other / Non-Disclosed,1.91,11



## Purchasing Analysis (Gender)

* Run basic calculations to obtain below values by gender
  * Purchase Count
  * Average Purchase Price  
  * Total Purchase Value
  * Average Purchase Total per Person by Gender
* Create a summary data frame to hold the results
* Display the summary data frame

In [7]:
by_gender = data.groupby('Gender')
purchase_count = pd.value_counts(data['Gender'])
#使用value_count算出每種value的總數
ave_purchase_price = by_gender.mean()['Price'].round(5)
total_value = by_gender.sum()['Price']
player_count = data.drop_duplicates('SN').groupby('Gender').count()['Purchase ID']
#使用drop_duplicates建立SN不重複的dataframe，再計算出各性別的購買數
avg_total_player = (total_value/player_count).round(2)
pd.DataFrame({'Purchase Count':purchase_count , 'Average Purchase Price':ave_purchase_price , 'Total Purchase Value':total_value , 'Avg Purchase Total per Person':avg_total_player})

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Purchase Total per Person
Female,113,3.20301,361.94,4.47
Male,652,3.01785,1967.64,4.07
Other / Non-Disclosed,15,3.34600,50.19,4.56


## Age Demographics

* Categorize players using the age groups. Hint: use pd.cut()
  * each group is for 4 years (i.e. &lt;10, 10-14, 15-19, etc.)
* Calculate the numbers and percentages by age group
* Create a summary data frame to hold the results
* round the percentage column to two decimal points
* Display Age Demographics Table

In [8]:
age_count = pd.value_counts(pd.cut(data.drop_duplicates('SN')['Age'] , [min(data['Age']-1),9,14,19,24,29,34,39,max(data['Age'])]))
#使用cut，bins的最小值和最大值設資料中最大值+1和最小值-1以計算到最大值和最小值
num_of_player = data.drop_duplicates('SN').shape[0]
ans = pd.DataFrame({'Percentage of Players':(100*age_count/num_of_player).round(2) , 'Total Count':age_count}).sort_index()
ans.index = ['<10' , '10-14' , '15-19' , '20-24' , '25-29' , '30-34' , '35-39' , '40+']
ans

,Percentage of Players,Total Count
<10,2.95,17
10-14,3.82,22
15-19,18.58,107
20-24,44.79,258
25-29,13.37,77
30-34,9.03,52
35-39,5.38,31
40+,2.08,12


## Purchasing Analysis (Age)

* The below each broken into groups of 4 years (i.e. &lt;10, 10-14, 15-19, etc.)
  * Purchase Count
  * Average Purchase Price
  * Total Purchase Value
  * Average Purchase Total per Person by Age Group
* Bin the purchase_data data frame by age
* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below
* Create a summary data frame to hold the results
* Display the summary data frame

In [9]:
as_interval = pd.cut(data.Age , [min(data.Age)-1 , 9,14,19,24,29,34,39,max(data.Age)+1] , labels = ['<10' , '10-14','15-19','20-24','25-29','30-34','35-39','>40'])
data['Age Interval'] = as_interval
#在dataframe中新增一value為包含當筆資料玩家的年齡的間距之col，'Age Interval'方便後續操作
age_count = pd.value_counts(as_interval)
ave_price_age = data.groupby('Age Interval')['Price'].mean()
#以年齡間距來做分組，分別計算mean、sum、count
total_pur_val_age = data.groupby('Age Interval')['Price'].sum()
num_age = data.drop_duplicates('SN').groupby('Age Interval').count()['Purchase ID']
aptpp = total_pur_val_age/num_age
pd.DataFrame({'Purchase Count':age_count , 'Average Purchase Price':ave_price_age , 'Total Purchase Value' : total_pur_val_age , 'Average Purchase Total per Person': aptpp}).round(2)


,Purchase Count,Average Purchase Price,Total Purchase Value,Average Purchase Total per Person
<10,23,3.35,77.13,4.54
10-14,28,2.96,82.78,3.76
15-19,136,3.04,412.89,3.86
20-24,365,3.05,1114.06,4.32
25-29,101,2.90,293.00,3.81
30-34,73,2.93,214.00,4.12
35-39,41,3.60,147.67,4.76
>40,13,2.94,38.24,3.19


## Top Spenders

* Identify the the top 5 spenders in the game by total purchase value (top-5 players with highest total purchase value), then list (in a table):
  * SN(screen name)
  * Purchase Count
  * Average Purchase Price
  * Total Purchase Value
* Run basic calculations to obtain the results in the table below
* Create a summary data frame to hold the results
* Sort the total purchase value column in descending order
* Display a preview of the summary data frame

In [10]:
purchase_value = {}
purchase_count = {}
#以SN為Key分別建立value為sum of purchase value和count的dictionary
for sn in set(data.SN):
    if sn in purchase_value:
        continue
    else:
        sum = data[data.SN==sn]['Price'].sum()
        p_count = data[data.SN==sn]['Price'].count()
        purchase_value[sn] = sum
        purchase_count[sn] = p_count
purchase_value = pd.Series(purchase_value)
purchase_count = pd.Series(purchase_count)
#轉為series讓兩資料能照著index排序
pd.DataFrame({'Purchase Count': purchase_count , 'Average Purchase Price':(purchase_value/purchase_count) , 'Total Purchase Value':purchase_value}).sort_values(by = 'Total Purchase Value' , ascending = False)[:5].round(2)

,Purchase Count,Average Purchase Price,Total Purchase Value
Lisosia93,5,3.79,18.96
Idastidru52,4,3.86,15.45
Chamjask73,3,4.61,13.83
Iral74,4,3.40,13.62
Iskadarya95,3,4.37,13.10


## Most Popular Items

* Top 5 most popular items by purchase count:
  * Item ID
  * Item Name
  * Purchase Count
  * Item Price
  * Total Purchase Value
* Retrieve the Item ID, Item Name, and Item Price columns
* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value
* Create a summary data frame to hold the results
* Sort the purchase count column in descending order
* Display a preview of the summary data frame

In [117]:
gb = data.groupby(['Item ID','Item Name'])
#照Item ID 和 Item Name分類，分別計算count、mean、sum
count_id = gb.count()['Purchase ID']
price = gb.mean()['Price']
total = gb.sum()['Price']
ans = pd.DataFrame({'Purchase Count':count_id , 'Item Price':price , 'Total Purchase Value':total})
ans.sort_values(by = 'Purchase Count' , ascending = False)[:5]

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,50.76
145,Fiery Glass Crusader,9,4.58,41.22
108,"Extraction, Quickblade Of Trembling Hands",9,3.53,31.77
82,Nirvana,9,4.90,44.10
19,"Pursuit, Cudgel of Necromancy",8,1.02,8.16


## Most Profitable Items

* Top 5 most profitable items by total purchase value:
  * Item ID
  * Item Name
  * Purchase Count
  * Item Price
  * Total Purchase Value
* Sort the above table by total purchase value in descending order
* Display a preview of the data frame

In [124]:
gb = data.groupby(['Item ID' , 'Item Name'])
pc = gb.count()['Price']
ip = gb.mean()['Price']
tpv = gb.sum()['Price']
ans = pd.DataFrame({'Purchase Count':pc , 'Item Price':ip , 'Total Purchase Value':tpv})
#和上題相同，改成sort by Total Purchase Value.
ans.sort_values(by = 'Total Purchase Value' , ascending = False)[:5]

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,50.76
82,Nirvana,9,4.90,44.10
145,Fiery Glass Crusader,9,4.58,41.22
92,Final Critic,8,4.88,39.04
103,Singed Scalpel,8,4.35,34.80
